In [1]:
# Import
import MetaTrader5 as mt5
import logging

from pandas import read_csv
from datetime import datetime, timedelta
import matplotlib as mpl
from matplotlib import pylab as plt
import pandas as pd

import numpy as np
import tensorflow as tf

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import pickle
from keras import backend as K

Using TensorFlow backend.


In [ ]:
d = datetime.now()
logging.basicConfig(filename='ForEx-Log' + d,
                            filemode='a',
                            format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                            datefmt='%H:%M:%S',
                            level=logging.DEBUG)

logging.info("Running FOREX TRADER ON", d)

In [2]:
#connect
server = 'MetaQuotes-Demo'
password = 'tzgypx4u'
account = 32331337
TIMEOUT = 600000

# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
    

authorized=mt5.login(account, password=password, server=server, timeout=TIMEOUT)
if authorized:
    # display trading account data 'as is'
    print(mt5.account_info())
    # display trading account data in the form of a list
    print("Show account_info()._asdict():")
    account_info_dict = mt5.account_info()._asdict()
    for prop in account_info_dict:
        print("  {}={}".format(prop, account_info_dict[prop]))
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))
 


AccountInfo(login=32331337, trade_mode=0, leverage=100, limit_orders=200, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=99880.87, credit=0.0, profit=-0.77, equity=99880.1, margin=100.0, margin_free=99780.1, margin_level=99880.1, margin_so_call=50.0, margin_so_so=30.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Jonathan Noirot', server='MetaQuotes-Demo', currency='EUR', ...)
Show account_info()._asdict():
  login=32331337
  trade_mode=0
  leverage=100
  limit_orders=200
  margin_so_mode=0
  trade_allowed=True
  trade_expert=True
  margin_mode=2
  currency_digits=2
  fifo_close=False
  balance=99880.87
  credit=0.0
  profit=-0.77
  equity=99880.1
  margin=100.0
  margin_free=99780.1
  margin_level=99880.1
  margin_so_call=50.0
  margin_so_so=30.0
  margin_initial=0.0
  margin_maintenance=0.0
  assets=0.0
  liabilities=0.0
  commission_blocked=0.0
  name=Jonat

In [3]:
# get currencies rates
currencies = ['EURCAD', 'EURCHF', 'EURGBP', 'EURJPY', 'EURUSD', 'EURCZK', 'EURDKK', 'EURHKD', 'EURHUF', 'EURMXN', 'EURNOK', 'EURNZD', 'EURPLN', 'EURRUB', 'EURRUR', 'EURSEK', 'EURTRY', 'EURZAR']
#currencies = ['EURCAD']

lookback = 12
date = datetime.now() + timedelta(hours=2)
date = date.replace(second=0, microsecond=0, minute=0)
print(date)
last_rates = {}

for curr in currencies:
    curr_rates = []
    curr_info = mt5.copy_rates_from(curr, mt5.TIMEFRAME_H1, date, lookback)
    #print(curr_info)
    for v in curr_info:
        print(curr, ' h: ', datetime.fromtimestamp(v[0]), ' rates ', v[4])
        curr_rates.append(v[4])
        
    last_rates[curr] = curr_rates


2020-07-22 20:00:00
EURCAD  h:  2020-07-22 09:00:00  rates  1.5514700000000001
EURCAD  h:  2020-07-22 10:00:00  rates  1.55106
EURCAD  h:  2020-07-22 11:00:00  rates  1.5505200000000001
EURCAD  h:  2020-07-22 12:00:00  rates  1.55146
EURCAD  h:  2020-07-22 13:00:00  rates  1.5523099999999999
EURCAD  h:  2020-07-22 14:00:00  rates  1.55322
EURCAD  h:  2020-07-22 15:00:00  rates  1.5553
EURCAD  h:  2020-07-22 16:00:00  rates  1.5543900000000002
EURCAD  h:  2020-07-22 17:00:00  rates  1.55709
EURCAD  h:  2020-07-22 18:00:00  rates  1.55743
EURCAD  h:  2020-07-22 19:00:00  rates  1.55688
EURCAD  h:  2020-07-22 20:00:00  rates  1.55339
EURCHF  h:  2020-07-22 09:00:00  rates  1.0757
EURCHF  h:  2020-07-22 10:00:00  rates  1.07555
EURCHF  h:  2020-07-22 11:00:00  rates  1.07483
EURCHF  h:  2020-07-22 12:00:00  rates  1.07487
EURCHF  h:  2020-07-22 13:00:00  rates  1.0764
EURCHF  h:  2020-07-22 14:00:00  rates  1.07812
EURCHF  h:  2020-07-22 15:00:00  rates  1.07863
EURCHF  h:  2020-07-22 16:0

In [4]:
def prepareRates(last_rates, scalers):
    data_curr = {}
    for curr, val in last_rates.items():
        
        data = {}
        
        scaler = scalers[curr]
        print(len(val))
        
        dataset = np.reshape(val, (1, len(val)))
        scaled_data = scaler.transform(dataset)
                
        data['dataset'] = dataset
        data['scaler'] = scaler
        
        data_curr[curr] = data
        
        print('For ', curr)
        
        print('dataset.shape ', dataset.shape)
    return data_curr

In [5]:
def predictRates(data_curr, models):
    
    predictions = {}

    i = 0
    for curr, model in models.items():
        x_test = data_curr[curr]['dataset']
        X = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)
        
        prediction = model.predict(X)
        predictions[curr] = prediction.reshape(prediction.shape[0], 1)
        i += 1
    return predictions

In [6]:
with open('scalers', 'rb') as handle:
    scalers = pickle.load(handle)
    
data_curr = prepareRates(last_rates, scalers)

12
For  EURCAD
dataset.shape  (1, 12)
12
For  EURCHF
dataset.shape  (1, 12)
12
For  EURGBP
dataset.shape  (1, 12)
12
For  EURJPY
dataset.shape  (1, 12)
12
For  EURUSD
dataset.shape  (1, 12)
12
For  EURCZK
dataset.shape  (1, 12)
12
For  EURDKK
dataset.shape  (1, 12)
12
For  EURHKD
dataset.shape  (1, 12)
12
For  EURHUF
dataset.shape  (1, 12)
12
For  EURMXN
dataset.shape  (1, 12)
12
For  EURNOK
dataset.shape  (1, 12)
12
For  EURNZD
dataset.shape  (1, 12)
12
For  EURPLN
dataset.shape  (1, 12)
12
For  EURRUB
dataset.shape  (1, 12)
12
For  EURRUR
dataset.shape  (1, 12)
12
For  EURSEK
dataset.shape  (1, 12)
12
For  EURTRY
dataset.shape  (1, 12)
12
For  EURZAR
dataset.shape  (1, 12)


In [7]:
def loadModel(curr_key):
    i = 0
    models = {}
    for curr in curr_key:
        # load the model from disk
        filename = curr + '_model.sav'
        loaded_model = pickle.load(open(filename, 'rb'))
        models[curr] = loaded_model
        print('Loaded model: ', filename)
    return models

In [8]:
models = loadModel(currencies)
predictions = predictRates(data_curr, models)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Loaded model:  EURCAD_model.sav
Loaded model:  EURCHF_model.sav
Loaded model:  EURGBP_model.sav
Loaded model:  EURJPY_model.sav
Loaded model:  EURUSD_model.sav
Loaded model:  EURCZK_model.sav
Loaded model:  EURDKK_model.sav
Loaded model:  EURHKD_model.sav
Loaded model:  EURHUF_model.sav
Loaded model:  EURMXN_model.sav
Loaded model:  EURNOK_model.sav
Loaded model:  EURNZD_model.sav
Loaded model:  EURPLN_model.sav
Loaded model:  EURRUB_model.sav
Loaded model:  EURRUR_model.sav
Loaded model:  EURSEK_model.sav
Loaded model:  EURTRY_model.sav
Loaded model:  EURZAR_model.sav


In [88]:
print(predictions)

{'EURCAD': array([[1.5407041]], dtype=float32)}


In [89]:
def getActions(data_curr, predictions):

    A = {}
    print(data_curr)
    
    for curr, dataset in data_curr.items():        
        action = ''
        
        y = dataset['dataset']
        n = len(y)
        d_cur = y[0][-1] - y[0][-2]

        d_next = predictions[curr][0][0] - y[0][-1]
        
        print(d_cur, d_next)

        if d_cur > 0 and d_next < 0:
            action = 'BUY'

        elif d_cur < 0 and d_next > 0:
            action = 'SELL'
        A[curr] = action
    
    return A

In [90]:
A = getActions(data_curr, predictions)
print(A)

{'EURCAD': {'dataset': array([[1.55147, 1.55106, 1.55052, 1.55146, 1.55231, 1.55322, 1.5553 ,
        1.55439, 1.55709, 1.55743, 1.55688, 1.55339]]), 'scaler': MinMaxScaler()}}
-0.003489999999999993 -0.01268586887359624
{'EURCAD': ''}


In [80]:
def get_info(symbol):
    '''https://www.mql5.com/en/docs/integration/python_metatrader5/mt5symbolinfo_py
    '''
    # get symbol properties
    info=mt5.symbol_info(symbol)
    return info

def open_trade(action, symbol, lot, sl_points, tp_points, deviation):
    '''https://www.mql5.com/en/docs/integration/python_metatrader5/mt5ordersend_py
    '''
    # prepare the buy request structure
    ea_magic_number = 234000
    symbol_info = get_info(symbol)
    
    if action == 'BUY':
        trade_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    elif action =='SELL':
        trade_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
        
    point = mt5.symbol_info(symbol).point

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": trade_type,
        "price": price,
        "deviation": deviation,
        "magic": ea_magic_number,
        "comment": "sent by python",
        "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    # send a trading request
    result = mt5.order_send(request)        
    return result, request 

def close_trade(position_id, symbol, lot, deviation):
    '''https://www.mql5.com/en/docs/integration/python_metatrader5/mt5ordersend_py
    '''
    # create a close request
    ea_magic_number = 234000
    if action == 'BUY':
        trade_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    elif action =='SELL':
        trade_type = mt5.ORDER_TYPE_SELL
    price = mt5.symbol_info_tick(symbol).bid
        
    close_request={
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "position": position_id,
        "price": price,
        "deviation": deviation,
        "magic": ea_magic_number,
        "comment": "python script close",
        "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    # send a close request
    result=mt5.order_send(close_request)
    return result, close_request 

In [81]:
sl_points = 100
tp_points = 1000
deviation = 50
count_buy = 0
for curr, a in A.items():
    if a == 'BUY':
        count_buy += 1

In [82]:
for symbol, action in A.items():
    lot = 0.1

    if action == 'SELL':
        positions=mt5.positions_get(symbol=symbol)
        if positions==None:
            print("No positions on,", symbol,", error code={}".format(mt5.last_error()))
        elif len(positions)>0:
            print("Total positions on ", symbol," =",len(positions))
            # display all open positions
            for position in positions:
                print(position)
                if position.profit > 0 and position.sl != 0:
                    lot = position.volume
                    position_id = position.ticket

                    result, close_request  = close_trade(position_id, symbol, lot, deviation)
                    print('result:', result)
                    print('close_request:', close_request)
                    
    elif action == 'BUY':
        lot = 0.1
        result, request = open_trade(action, symbol, lot, sl_points, tp_points, deviation)
        print('result:', result)
        print('request:', request)
                    
mt5.shutdown()

Total positions on  EURCAD  = 1
TradePosition(ticket=662418991, time=1595445572, time_msc=1595445572782, time_update=1595445572, time_update_msc=1595445572782, type=0, magic=234000, identifier=662418991, reason=3, volume=0.1, price_open=1.55229, sl=0.0, tp=1.56229, price_current=1.55203, swap=0.0, profit=-1.6800000000000002, symbol='EURCAD', comment='sent by python', external_id='')
